In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import userdata
import pandas as pd
import json
import requests
import openpyxl

##Extract the list of questions from the Excel file into a JSON list

In [ ]:
file_path = '/content/drive/My Drive/LLM Evaluations/llm_evals_test_data_with_context.xlsx'
# Read the Excel file into a pandas DataFrame
df = pd.read_excel(file_path, engine='openpyxl')
faqs = df.to_dict(orient='records')
print(faqs)

## Launch a user session on **Voiceflow**
Start a new chat session with the Voiceflow Chatbot.

In [ ]:
api_url = "https://general-runtime.voiceflow.com/state/user/user69/interact?logs=off"
api_key = userdata.get('VOICEFLOW_API_KEY')
api_headers = {'content-Type': 'application/json','accept':'application/json','Authorization':api_key}

def launch_session():
    data = {
      "action": {
          "type": "launch"
      },
      "config": {
        "tts": False,
        "stripSSML": True,
        "stopAll": True,
        "excludeTypes": [
            "block",
            "debug",
            "flow"
          ]
        }
    }
    response = requests.post(api_url, json=data, headers=api_headers)
    print(response.text)

launch_session()

## Get answers from Voiceflow

In [ ]:
def get_answer(question):
  print(f"Question: {question}")
  data = {
      "action": {
          "type": "text",
          "payload": question
      },
      "config": {
        "tts": False,
        "stripSSML": True,
        "stopAll": True,
        "excludeTypes": [
            "block",
            "debug",
            "flow"
          ]
        }
    }

  response = requests.post(api_url, json=data, headers=api_headers)

  # if request fails return 'Error', else return response
  if response.status_code != 200:
    return f"Error:{response.text}"

  json_array = response.json()
  print(f"Response JSON: {json_array}")
  # Filter json_array to extract items where type = text
  text_responses = [obj for obj in json_array if obj['type'] == 'text']
  answer = text_responses[0]['payload']['message']
  print(f"Answer: {answer}")
  return answer

In [ ]:
# Add answers from the chatbot to the JSON array containing questions
def add_answers_to(json_list):
    headers = {'Content-Type': 'application/json'}

    for json_obj in faqs:
        question = json_obj.get('question')
        if question:
                voiceflow_response = get_answer(question)

                # Add the response to the JSON object
                json_obj['response'] = voiceflow_response

    return json_list

json_list_with_answers = add_answers_to(faqs)
# Print the processed JSON objects
json_string = json.dumps(json_list_with_answers, indent=2)

print(json_string)

## Generate an Excel file from the updated JSON

In [ ]:
output_file_path = '/content/drive/My Drive/LLM Evaluations/llm_evals_test_data_with_response.xlsx'
df = pd.DataFrame(json_list_with_answers)
df.to_excel(output_file_path, index=False, engine='openpyxl')